<div >
<img src = "figs/ans_banner_1920x200.png" />
</div>

# Caso-taller:  Recomendando el Blog de  Hernán Casciari 


[Hernán Casciari](https://hernancasciari.com/#bio), es un escritor argentino, que escribe blog posts con cuentos e historias  relacionadas con el futbol, su vida, infancia, y relaciones familiares con toques de ficción. Este [blog](https://hernancasciari.com/blog/) es  tan interesantes que en 2005 fue premiado como “El mejor blog del mundo” por Deutsche Welle de Alemania. 

El objetivo de este caso-taller es construir un sistema de recomendación basado en los contenidos de los posts utilizando similitud de las palabras usadas o temas de los cuentos.

## Instrucciones generales

1. Para desarrollar el *cuaderno* primero debe descargarlo.

2. Para responder cada inciso deberá utilizar el espacio debidamente especificado.

3. La actividad será calificada sólo si sube el *cuaderno* de jupyter notebook con extensión `.ipynb` en la actividad designada como "Revisión por el compañero."

4. El archivo entregado debe poder ser ejecutado localmente por los pares. Sea cuidadoso con la especificación de la ubicación de los archivos de soporte, guarde la carpeta de datos  en la misma ruta de acceso del cuaderno, por ejemplo: `data`.

## Desarrollo


### 1. Carga de datos 

En la carpeta `data` se encuentran el archivo `blog_casciari.csv` con el título, la fecha de publicación, y el contenido de los cuentos publicados en el blog  de sr. Casciari. Cargue estos datos en su *cuaderno* y reporte brevemente el contenido de la base.
   

In [28]:
# Importamos las librerías necesarias
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import correlation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora, models
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

In [4]:
# Cargamos los datos desde el archivo CSV
file_path = 'blog_casciari.csv'
df = pd.read_csv(file_path)

# Mostramos las columnas disponibles en el dataset
print("Columnas disponibles en el dataset:")
print(df.columns)

# Mostramos las primeras filas del dataframe para tener una idea general del contenido
print("\nPrimeras 5 filas del dataset:")
print(df.head())

# Mostramos un resumen básico del dataset
print("\nInformación del dataset:")
df.info()


Columnas disponibles en el dataset:
Index(['titulo', 'fecha', 'cuento'], dtype='object')

Primeras 5 filas del dataset:
                       titulo    fecha  \
0            El rincón blanco  1/11/08   
1  Mínimos avances en la cama  1/24/08   
2                  Don Marcos  2/19/08   
3              Los dos rulfos  3/26/08   
4   La noticia no es el perro  4/15/08   

                                              cuento  
0  De pronto yo estaba en el hogar donde pasé la ...  
1  Menos la cama, todo ha mejorado en este mundo....  
2  Dos veces, y no una, mi abuelo materno me ayud...  
3  A su regreso de México, mi amigo Comequechu no...  
4  De repente, un video de You Tube recibe un mil...  

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   titulo  520 non-null    object
 1   fecha   520 non-null    object
 2   cuento  520 non-nu

Para traer los datos, cargue el archivo blog_casciari.csv utilizando la librería pandas y revise el contenido del dataset, donde primero se muestran las columnas disponibles en el archivo para identificar los nombres correctos, luego vemos las primeras filas del dataframe para obtener una vista previa de los datos y proporcionamos un resumen general del dataset con el método .info().

El dataset cargado contiene 520 entradas y 3 columnas: 'titulo', 'fecha', y 'cuento'.

### 2. Homogenización de textos

Para cumplir con el objetivo de generar recomendaciones en esta sección debe preparar los posts para poder ser utilizados en su sistema de recomendación. Para ello, "limpie" y "tokenize" cada uno de los cuentos, describiendo detalladamente los pasos que realizo y si transformó o eliminó ciertas palabras. Para asistirlo en la tarea he creado listas de *stopwords* que están disponibles en la carpeta `data`. En su procedimiento ilustre la limpieza con el cuento 'La venganza del metegol'. (En su limpieza recuerde que el objetivo es generar recomendaciones a partir de la similitud de las palabras o temas de los cuentos)

In [9]:
# Cargar la lista de stopwords
file_path = 'stopwords_taller.csv'
custom_stopwords  = pd.read_csv(file_path)
custom_stopwords

,ahora
0,alejandro
1,alex
2,alfonso
3,alguien
4,allí
...,...
163,éstos
164,českomoravský
165,české
166,šeredova


In [10]:
# Aseguramos que las stopwords estén en minúsculas para una limpieza efectiva
custom_stopwords = [word.lower() for word in custom_stopwords]

In [11]:
# Función para limpiar y tokenizar el texto
def clean_and_tokenize(text):
    # Convertimos todo el texto a minúsculas
    text = text.lower()
    
    # Eliminamos caracteres no deseados como signos de puntuación, números y caracteres especiales
    text = re.sub(r'[^a-zñáéíóúü\s]', '', text)
    
    # Tokenizamos el texto (convertirlo en una lista de palabras)
    tokens = word_tokenize(text)
    
    # Eliminamos stopwords (palabras irrelevantes)
    tokens = [word for word in tokens if word not in custom_stopwords]
    
    return tokens

In [13]:
# Aplicamos la función a todos los cuentos
df['tokens'] = df['cuento'].apply(clean_and_tokenize)

# Mostramos las primeras filas con la columna de tokens
print("\nCuentos tokenizados:")
print(df[['titulo', 'tokens']].head())


Cuentos tokenizados:
                       titulo  \
0            El rincón blanco   
1  Mínimos avances en la cama   
2                  Don Marcos   
3              Los dos rulfos   
4   La noticia no es el perro   

                                              tokens  
0  [de, pronto, yo, estaba, en, el, hogar, donde,...  
1  [menos, la, cama, todo, ha, mejorado, en, este...  
2  [dos, veces, y, no, una, mi, abuelo, materno, ...  
3  [a, su, regreso, de, méxico, mi, amigo, comequ...  
4  [de, repente, un, video, de, you, tube, recibe...  


In [14]:
# Extraemos el cuento de 'La venganza del metegol' para el ejemplo
cuento_example = df[df['titulo'] == 'La venganza del metegol']['cuento'].values[0]

# Aplicamos la función de limpieza y tokenización al cuento
tokens_cuento = clean_and_tokenize(cuento_example)

# Mostramos los primeros tokens resultantes
print("Tokens del cuento 'La venganza del metegol':")
print(tokens_cuento[:20])  # Mostramos solo los primeros 20 tokens para ilustrar el resultado

# Mostramos la longitud del cuento original y del texto tokenizado para comparación
print(f"\nLongitud original del cuento: {len(cuento_example.split())} palabras")
print(f"Longitud tras la limpieza: {len(tokens_cuento)} palabras")

Tokens del cuento 'La venganza del metegol':
['el', 'mes', 'pasado', 'me', 'invitaron', 'a', 'presentar', 'un', 'libro', 'en', 'buenos', 'aires', 'y', 'como', 'era', 'un', 'libro', 'sobre', 'fútbol', 'al']

Longitud original del cuento: 1137 palabras
Longitud tras la limpieza: 1128 palabras


Para este punto se carga el archivo con las stopwords sugeridas, donde nos aseguramos que todas las palabras estén en minúsculas para facilitar la comparación con los tokens, luego se crea la función de limpieza y tokenización donde se convierte el texto a minúsculas para uniformar el formato, se eliminan los caracteres especiales, signos de puntuación y números que no son útiles para el análisis semántico.

Finalmente Tokenizamos el texto en palabras utilizando word_tokeniz y eliminamos todas las palabras que están en la lista de stopwords para reducir el ruido y enfocarnos en palabras significativas.


Transformación o eliminación de palabras:
Todas las palabras se transforman a minúsculas.
Se eliminan los caracteres especiales y números.
Se eliminan las palabras irrelevantes según la lista de stopwords.

A modo de revision se toma el cuento 'La venganza del metegol' y se hace la limpieza y tokenización de su contenido donde se observa que se paso de 1137 palabras antes a 1128 ahora luego de este paso.

### 3. Generando Recomendaciones

En esta sección nos interesa generar recomendaciones de cuentos en el blog a un usuario que leyó 'La venganza del metegol'. Para ello vamos a utilizar distintas estrategias.

#### 3.1. Recomendaciones basadas en contenidos

##### 3.1.1. Genere 5 recomendaciones de más recomendada (1) a menos recomendada (5) para el cuento 'La venganza del metegol' usando en la distancia de coseno donde el texto este vectorizado por `CountVectorizer`. Explique el procedimiento que realizó y como ordenó las recomendaciones.

In [16]:
# Vectorización de los textos utilizando CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['cuento'])

# Obtenemos el índice del cuento 'La venganza del metegol'
index_cuento_ejemplo = df[df['titulo'] == 'La venganza del metegol'].index[0]

# Calculamos la similitud de coseno entre el cuento 'La venganza del metegol' y los demás cuentos
cosine_similarities = cosine_similarity(X[index_cuento_ejemplo], X).flatten()

# Ordenamos los cuentos por similitud, excluyendo el propio cuento de referencia
similar_cuentos_indices = cosine_similarities.argsort()[::-1][1:6]  # Tomamos los 5 más similares

# Mostramos las 5 recomendaciones
print("Top 5 recomendaciones para 'La venganza del metegol':")
for i, idx in enumerate(similar_cuentos_indices):
    print(f"{i+1}. {df['titulo'].iloc[idx]} (Similitud: {cosine_similarities[idx]:.4f})")


Top 5 recomendaciones para 'La venganza del metegol':
1. Cuento con bruja y tramontina (Similitud: 0.9132)
2. Primer asalto (Similitud: 0.9111)
3. Lado A: música ligera (Similitud: 0.9081)
4. La desgracia venía en sobres papel madera (Similitud: 0.9070)
5. Electrodomésticos (Similitud: 0.9041)


Para generar las recomendaciones basadas en contenidos, vectoricé los cuentos usando CountVectorizer, que convierte el texto en una matriz de frecuencias de palabras, luego calculé la similitud de coseno entre el cuento 'La venganza del metegol' y los demás cuentos para medir qué tan similares son en cuanto a las palabras que usan y finalmente, ordené las recomendaciones de mayor a menor similitud de coseno, seleccionando los 5 cuentos con mayor similitud, excluyendo el propio cuento de referencia, donde podemos ver que el cuento con mayor similitud es Cuento con bruja y tramontina (Similitud: 0.9132), seguido por Primer asalto (Similitud: 0.9111),

##### 3.1.2. Genere 5 recomendaciones de más recomendada (1) a menos recomendada (5) para  el cuento 'La venganza del metegol' usando nuevamente la distancia de coseno, pero ahora vectorice el texto usando `TF-IDFVectorizer`. Explique el procedimiento que realizó y como ordenó las recomendaciones. Compare con los resultados del punto anterior y explique sus similitudes y/o diferencias.

In [22]:
# Vectorización de los textos utilizando TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(df['cuento'])

# Obtenemos el índice del cuento 'La venganza del metegol'
index_cuento_ejemplo = df[df['titulo'] == 'La venganza del metegol'].index[0]

# Calculamos la similitud de coseno entre el cuento 'La venganza del metegol' y los demás cuentos
cosine_similarities_tfidf = cosine_similarity(X_tfidf[index_cuento_ejemplo], X_tfidf).flatten()

# Ordenamos los cuentos por similitud, excluyendo el propio cuento de referencia
similar_cuentos_indices_tfidf = cosine_similarities_tfidf.argsort()[::-1][1:6]

# Mostramos las 5 recomendaciones
print("Top 5 recomendaciones para 'La venganza del metegol' (usando TF-IDF):")
for i, idx in enumerate(similar_cuentos_indices_tfidf):
    print(f"{i+1}. {df['titulo'].iloc[idx]} (Similitud: {cosine_similarities_tfidf[idx]:.4f})")


Top 5 recomendaciones para 'La venganza del metegol' (usando TF-IDF):
1. Cuento con bruja y tramontina (Similitud: 0.4968)
2. Gaussian blur (Similitud: 0.4960)
3. Dice el Chiri, dice el Gordo (Similitud: 0.4956)
4. La desgracia venía en sobres papel madera (Similitud: 0.4794)
5. Matar la crisis a volantazos (Similitud: 0.4677)


Para este caso de generar las recomendaciones con TF-IDFVectorizer, transformé los cuentos en vectores ponderados según la relevancia de las palabras, en lugar de simplemente contar su frecuencia, calculé la similitud de coseno entre el vector del cuento 'La venganza del metegol' y los vectores de los otros cuentos, y ordené los resultados de mayor a menor similitud, seleccionando los 5 cuentos más similares.

Comparado con el método de CountVectorizer, que contaba palabras sin ponderar su relevancia, TF-IDF ofrece recomendaciones más ajustadas al dar más peso a palabras distintivas y menos a términos comunes, lo que hizo que aunque el cuento con mayor similitud siga siendo Cuento con bruja y tramontina (Similitud: 0.4968) y la desgracia venía en sobres papel madera (Similitud: 0.4794) tambien sea recomendado, el segundo,tercero y quinto puesto si cambiaron.

##### 3.1.3. Genere 5 recomendaciones de más recomendada (1) a menos recomendada (5) para el cuento 'La venganza del metegol' usando el texto vectorizado por `TF-IDFVectorizer` y la correlación como medida de similitud. Explique el procedimiento que realizó y como ordenó las recomendaciones. Compare con los resultados de los puntos anteriores y explique sus similitudes y/o diferencias.

In [21]:
# Vectorización de los textos utilizando TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(df['cuento'])

# Obtenemos el índice del cuento 'La venganza del metegol'
index_cuento_ejemplo = df[df['titulo'] == 'La venganza del metegol'].index[0]

# Calculamos la correlación entre el cuento 'La venganza del metegol' y los demás cuentos
correlation_scores = []
for i in range(X_tfidf.shape[0]):
    if i != index_cuento_ejemplo:
        correlation_score = 1 - correlation(X_tfidf[index_cuento_ejemplo].toarray().flatten(), X_tfidf[i].toarray().flatten())
        correlation_scores.append((i, correlation_score))

# Ordenamos los cuentos por correlación
sorted_correlation_scores = sorted(correlation_scores, key=lambda x: x[1], reverse=True)
top_5_indices = [index for index, score in sorted_correlation_scores[:5]]

# Mostramos las 5 recomendaciones
print("Top 5 recomendaciones para 'La venganza del metegol' (usando Correlación):")
for i, idx in enumerate(top_5_indices):
    print(f"{i+1}. {df['titulo'].iloc[idx]} (Correlación: {sorted_correlation_scores[i][1]:.4f})")


Top 5 recomendaciones para 'La venganza del metegol' (usando Correlación):
1. Cuento con bruja y tramontina (Correlación: 0.4927)
2. Gaussian blur (Correlación: 0.4920)
3. Dice el Chiri, dice el Gordo (Correlación: 0.4915)
4. La desgracia venía en sobres papel madera (Correlación: 0.4752)
5. Matar la crisis a volantazos (Correlación: 0.4633)


Para generar las recomendaciones usando correlación con TF-IDFVectorizer, convertí los cuentos en vectores ponderados y calculé la correlación entre el vector del cuento 'La venganza del metegol' y los vectores de los demás cuentos, luego ordené las recomendaciones según la correlación más alta, destacando los cuentos con patrones de términos similares y comparado con la similitud de coseno, la correlación también revela cuentos con significativos patrones de palabras, pero puede diferir en el ranking debido a su forma diferente de medir similitud. 

Los resultados muestran similitudes en la inclusión de cuentos con términos relevantes, pero las diferencias en el orden reflejan las variaciones en cómo cada métrica interpreta la relación entre textos, nuevamente el cuento "Cuento con bruja y tramontina" ocupa el primer lugar y los demás cuentos son similares a los obtenidos en el punto anterior.

##### 3.2. Recomendaciones basadas en temas

Usando modelado de temas con LDA, encuentre los temas subyacentes en el blog. Explique como eligió el numero óptimo de temas. Utilizando el tema asignado al cuento 'La venganza del metegol' y la probabilidad de pertenecer a este tema genere 5 recomendaciones de más recomendada (1) a menos recomendada (5) para este cuento. Explique el procedimiento que realizó. Compare con los resultados encontrados anteriormente y explique sus similitudes y/o diferencias. (Esto puede tomar mucho tiempo y requerir mucha capacidad computacional, puede aprovechar los recursos de [Google Colab](https://colab.research.google.com/))


In [37]:
import pandas as pd
import numpy as np
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# Preprocesamiento de texto
def preprocess_text(text):
    
    return text.lower().split()  # Tokenización básica por espacios

# Aplicar el preprocesamiento
texts = df['cuento'].map(preprocess_text)

# Crear un diccionario y corpus para LDA
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Evaluar modelos LDA con diferentes números de temas
num_topics_range = [5, 10, 15, 20]
coherence_scores = []
perplexities = []

for num_topics in num_topics_range:
    lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)
    
    # Calcular la coherencia del modelo
    coherence_model = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    coherence_scores.append((num_topics, coherence_score))

    # Calcular la perplejidad del modelo
    perplexity = lda_model.log_perplexity(corpus)
    perplexities.append((num_topics, perplexity))

# Elegir el mejor número de temas basado en la coherencia
best_n_topics_coherence = max(coherence_scores, key=lambda x: x[1])[0]

# Entrenar el modelo LDA con el número óptimo de temas
lda_best = models.LdaModel(corpus, num_topics=best_n_topics_coherence, id2word=dictionary, passes=10)

# Visualización del modelo LDA (opcional, si tienes pyLDAvis instalado)
# pyLDAvis.save_html(pyLDAvis.gensim.prepare(lda_best, corpus, dictionary), 'lda_visualization.html')

# Asignación de temas y cálculo de probabilidades
# Obtener las probabilidades de pertenencia a cada tema para cada cuento
topic_probabilities = np.array([[prob[1] for prob in lda_best.get_document_topics(doc, minimum_probability=0)] for doc in corpus])

# Obtener el índice del cuento 'La venganza del metegol'
index_cuento_ejemplo = df[df['titulo'] == 'La venganza del metegol'].index[0]

# Identificar el tema principal al que pertenece el cuento 'La venganza del metegol'
tema_ejemplo = topic_probabilities[index_cuento_ejemplo].argmax()

# Selección de los 5 cuentos más similares basados en la probabilidad de pertenencia al mismo tema
similarity_scores = [(i, topic_probabilities[i][tema_ejemplo]) for i in range(len(df)) if i != index_cuento_ejemplo]
sorted_similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
top_5_indices = [index for index, score in sorted_similarity_scores[:5]]

# Mostrar las 5 recomendaciones
print("Top 5 recomendaciones para 'La venganza del metegol' (usando LDA):")
for i, idx in enumerate(top_5_indices):
    print(f"{i+1}. {df['titulo'].iloc[idx]} (Probabilidad de pertenencia al tema: {sorted_similarity_scores[i][1]:.4f})")


Top 5 recomendaciones para 'La venganza del metegol' (usando LDA):
1. Ropa sucia (Probabilidad de pertenencia al tema: 0.9998)
2. 10.6 segundos (Probabilidad de pertenencia al tema: 0.9998)
3. Escupir el asado (Probabilidad de pertenencia al tema: 0.9998)
4. Papelitos (Probabilidad de pertenencia al tema: 0.9998)
5. Basdala (Probabilidad de pertenencia al tema: 0.9998)


En esta parte, utilicé el modelado de temas con LDA para analizar los cuentos y encontrar los temas subyacentes. Preprocesé los textos, creando un diccionario y un corpus para entrenar varios modelos LDA con diferentes números de temas. Elegí el mejor modelo basándome en la coherencia del mismo y luego, usando el tema asignado al cuento "La venganza del metegol", generé 5 recomendaciones de cuentos similares. Estas recomendaciones están ordenadas según su probabilidad de pertenencia al mismo tema, permitiéndome identificar las historias más relacionadas temáticamente.

Utilizando el modelo LDA para recomendar cuentos similares a *'La venganza del metegol'*, obtuvimos resultados que muestran alta coherencia temática, con todos los cuentos recomendados teniendo una probabilidad de pertenencia al tema de 0.9998. Esto indica que estos cuentos son altamente relevantes para el tema del cuento de referencia, sugiriendo que el modelo ha capturado efectivamente el tema dominante. En comparación con métodos basados en `CountVectorizer` y `TF-IDF`, que se centran en la similitud textual directa, LDA ofrece recomendaciones basadas en temas subyacentes, lo que puede ser útil para explorar contenido relacionado en un contexto temático más amplio.


### 4 Recomendaciones generales

De acuerdo con los resultados encontrados, en su opinión ¿qué procedimiento generó las mejores recomendaciones para la entrada elegida? ¿Cómo implementaría una evaluación objetiva de estas recomendaciones? Justifique su respuesta.

*Recomendaciones*

De acuerdo con los resultados obtenidos, el procedimiento de Latent Dirichlet Allocation (LDA) generó las recomendaciones más coherentes para el cuento *'La venganza del metegol'*. Esto se debe a que LDA identificó un tema dominante al que todos los cuentos recomendados tienen una alta probabilidad de pertenencia, sugiriendo que estos cuentos están bien alineados temáticamente con el cuento de referencia, esto ya que LDA ofrece una perspectiva más abstracta al centrarse en temas subyacentes, proporcionando recomendaciones basadas en la coherencia temática general en lugar de coincidencias textuales específicas. Esto puede ser más útil para descubrir contenido relevante que comparte contextos similares, lo cual es valioso en sistemas de recomendación basados en temas.

Los métodos basados en `CountVectorizer` y `TF-IDFVectorizer`, aunque son útiles para medir similitudes textuales exactas, pueden no capturar el contexto temático completo, resultando en recomendaciones que se centran más en la similitud de palabras que en el significado subyacente del contenido.

Evaluación Objetiva:

1.Utilizar métricas como la coherencia de temas para evaluar la calidad de los temas generados por LDA. La coherencia mide la consistencia y la interpretabilidad de los temas generados. Se pueden usar medidas como `coherence_score` para cuantificar cuán bien los términos en cada tema están relacionados entre sí.

2.Evaluación Humana:Realizar una evaluación manual de las recomendaciones para comprobar si las recomendaciones generadas son realmente relevantes para el cuento de referencia en términos de contenido y tema. Esto puede implicar una revisión cualitativa de las recomendaciones para confirmar su relevancia y coherencia.

3.Comparación de Métodos: Comparar los resultados obtenidos con LDA frente a `CountVectorizer` y `TF-IDFVectorizer` en términos de relevancia y utilidad. Esto puede implicar medir la precisión de las recomendaciones al evaluar la satisfacción del usuario o la relevancia temática de las recomendaciones.

4. Retroalimentación del Usuario:Implementar un sistema de retroalimentación en el que los usuarios puedan calificar la relevancia de las recomendaciones. Esto proporciona datos reales sobre la efectividad del sistema de recomendación desde la perspectiva del usuario final.

En resumen, **LDA** parece ser el mejor método para generar recomendaciones temáticamente coherentes y relevantes. Para una evaluación objetiva, se deben utilizar métricas de coherencia de temas, evaluaciones manuales, comparaciones entre métodos y retroalimentación de usuarios para asegurar la efectividad del sistema de recomendación.